In [4]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.spatial.distance as dist
%matplotlib inline
import pylab
import json

from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
df_users = pd.read_json('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/user.json', lines=True)

In [9]:
df_users.head()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,3.80,5174,284,5174,5175,78,299,1435,7829,7397,...,16856,"[2014, 2016, 2013, 2011, 2012, 2015, 2010, 2017]",209,"[M19NwFwAXKRZzt8koF11hQ, QRcMZ8pJJBBZaKubHOoMD...",16605,Cin,272,17019,lsSiIjAKVl-QRxKjRErBeg,2010-07-13
1,3.94,1556,211,1556,1285,101,134,1295,162,2134,...,40110,"[2014, 2017, 2011, 2012, 2015, 2009, 2013, 200...",835,"[eoSSJzdprj3jxXyi94vDXg, QF0urZa-0bxga17ZeY-9l...",10882,Andrea,2559,83681,om5ZiponkpRqUNa3pVPiRg,2006-01-18
2,4.72,15,1,15,5,0,1,11,8,20,...,55,[],17,"[Oa84FFGBw1axX8O6uDkmqg, SRcWERSl4rhm-Bz9zN_J8...",4,Gabe,277,45,-lGwMGHMC_XihFJNKCJNRg,2014-10-31
3,3.76,9,0,9,1,0,1,4,1,11,...,4,"[2016, 2017]",11,"[96DJovjKAtExnyBZRfniPQ, Djczq-bRn761U4FW77NUt...",4,Leah,436,15,D-ydMTPGWXTVm4_jjp0k9g,2013-04-01
4,4.23,276,0,276,59,0,8,51,169,386,...,6006,"[2017, 2016]",49,"[iN0A6QIrEFYoSGHFaknh8Q, B2HDoWNIzLlon0IhS1cmD...",360,Juan,921,9152,PcvbBOCOcs6_suRDH7TSTg,2012-08-16


In [10]:
#Reading big file line by line, taking first 1000 reviews
c=0
s = open('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/review_sub.json','w',encoding='utf8')
with open('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/review.json', encoding='utf8') as f:
    for line in f:
        c=c+1
        s.write(line)
        if(c == 100000):
            break

In [11]:
df_reviews = pd.read_json('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/review_sub.json', lines=True)

In [12]:
df_reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,uYHaNptLzDLoV_JZ_MuzUA,0,2016-07-12,0,VfBHSwC5Vz_pbFluy07i9Q,5,My girlfriend and I stayed here for 3 nights a...,0,cjpdDjZyprfyDG3RlkVG3w
1,uYHaNptLzDLoV_JZ_MuzUA,0,2016-10-02,0,3zRpneRKDsOPq92tq7ybAA,3,If you need an inexpensive place to stay for a...,0,bjTcT8Ty4cJZhEOEo01FGA
2,uYHaNptLzDLoV_JZ_MuzUA,0,2015-09-17,0,ne5WhI1jUFOcRn-b-gAzHA,3,Mittlerweile gibt es in Edinburgh zwei Ableger...,0,AXgRULmWcME7J6Ix3I--ww
3,uYHaNptLzDLoV_JZ_MuzUA,0,2016-08-21,0,llmdwOgDReucVoWEry61Lw,4,Location is everything and this hotel has it! ...,0,oU2SSOmsp_A8JYI7Z2JJ5w
4,uYHaNptLzDLoV_JZ_MuzUA,0,2013-11-20,0,DuffS87NaSMDmIfluvT83g,5,gute lage im stadtzentrum. shoppingmeile und s...,0,0xtbPEna2Kei11vsU-U2Mw


In [13]:
df_bus = pd.read_json('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/business.json', lines=True)

In [14]:
res_mer = pd.merge(df_bus,df_reviews, on='business_id')

In [15]:
res_mer.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,...,stars_x,state,cool,date,funny,review_id,stars_y,text,useful,user_id
0,2815 S Alma School Rd,{},gqv6zoGHSeTsNKynhjI-Ng,"[Departments of Motor Vehicles, Public Service...",Mesa,{},1,33.363863,-111.858118,Desert Title Service,...,3.5,AZ,0,2017-06-10,0,I5A5eQhO3KQYsUi3cV2r6g,5,Really great experience. My Wife and I called...,0,kpPdmI83k3JnaC_qIw-8sg
1,2815 S Alma School Rd,{},gqv6zoGHSeTsNKynhjI-Ng,"[Departments of Motor Vehicles, Public Service...",Mesa,{},1,33.363863,-111.858118,Desert Title Service,...,3.5,AZ,0,2017-07-12,0,ULl_yZN68q5NTsXia9tXGQ,5,"Great experience. Andrea helped me out, she wa...",0,jXtb96mzeyze_IPZnxzfBw
2,2815 S Alma School Rd,{},gqv6zoGHSeTsNKynhjI-Ng,"[Departments of Motor Vehicles, Public Service...",Mesa,{},1,33.363863,-111.858118,Desert Title Service,...,3.5,AZ,0,2016-02-27,0,D9RLzMQ-NRqPb4S0y74O_A,2,"It is much faster than going to MVD, which is ...",0,aDV_OFWXV5FUttDV1ixfKA
3,2815 S Alma School Rd,{},gqv6zoGHSeTsNKynhjI-Ng,"[Departments of Motor Vehicles, Public Service...",Mesa,{},1,33.363863,-111.858118,Desert Title Service,...,3.5,AZ,0,2015-08-12,0,ydECcuKRpYkaiWlMe0u_LA,1,Worst service from any business of any kind. I...,0,pR7c7cTteRtPG4PW7zRZdQ
4,2815 S Alma School Rd,{},gqv6zoGHSeTsNKynhjI-Ng,"[Departments of Motor Vehicles, Public Service...",Mesa,{},1,33.363863,-111.858118,Desert Title Service,...,3.5,AZ,0,2015-03-09,0,oAXAdK82r1kVqJH5OxQ6Vw,1,Fast service but the blonde older lady was ver...,0,aWLnxC4iaFX57kCfKOPdag


In [16]:
df = res_mer.pivot(index = 'user_id', columns ='business_id', values='stars_y')
df.head()

business_id,--8LPVSo5i0Oo61X01sV9A,--DdmeR16TRb3LsjG0ejrQ,-050d_XIor1NpCuWkbIVaQ,-1dMU2kz5AhTC6N1W9xwuQ,-2Arz8twKJmxHMS3Sv-TeA,-36lZCEYtRfW0ZR4Nsem5Q,-3EqqxYhmi3G0QG1-zPyCg,-5JxNFUpJIFEWTzp4OUcIw,-6BsCHv7TOzDe1vaxXU7wg,-6covFhiJGaI1Vt0OuYMcw,...,zoq5SERVEDBd7MaKPjlLJA,zpC-0J9p7V26ZZAVses64A,zqHCzyOGQiNY9Co_G7-Bow,ztu1Wrxe5USdQ0iAum-xvw,zu969Qo6R4w17ncmDCQ5rg,zuCu5ff06l4Pe33qYe58MA,zvJY4MXpx7DEDTuZFbLYYA,zvsJSNdg9UXEIL6FKiGa3Q,zw4Legbcu018p5WcZ74iWA,zzDJg87wUzGWPXesYEnFkQ
user_id,,,,,,,,,,,,,,,,,,,,,
---1lKK3aKOuomHnwAkAow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--104qdWvE99vaoIsj9ZJQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--BumyUHiO_7YsHurb9Hkw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--Nnm_506G_p8MxAOQna5w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--P-Qvza7AED8gnDrZkMgA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
items=df.columns.values

In [18]:
mat = df.as_matrix().astype(np.float)

In [19]:
# centralizing to mean zero
sum =0.0
count =0
for i in range(len(mat)):
    sum = 0.0
    count =0
    for j in range(len(mat[0])):
        if(math.isnan(mat[i][j])):
            continue
        sum += mat[i][j]
        count += 1
    avg = sum/count
    for j in range(len(mat[0])):
        if(math.isnan(mat[i][j])):
            continue
        mat[i][j] -= avg

In [20]:
ind = np.where(np.isnan(mat))
mat[ind] = 0

In [21]:
mat

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [22]:
#creating sparse matrix
sp_mat = sp.csr_matrix(mat)

In [23]:
print("Number of users:", len(mat))
print("Number of items:", len(mat[0]))

Number of users: 78276
Number of items: 4224


In [24]:
i = 0
for i in range(len(mat)):
    c = 0
    for j in range(len(mat[0])):
        if mat[i][j] > 0:
            c = c+1
    if c >4:
        break
print(i)

320


In [34]:
# List of indices of similar users
similar_users = []
similarity = []
def cosine_similarity(user):
    for i in range(len(mat)):
        cs = 1 - dist.cosine(mat[i],user)
        if cs > 0:
            similarity.append(cs)
            similar_users.append(i)

In [35]:
#calling method
#Let new user be mat[320]
cosine_similarity(mat[320])

/home/gautam/.local/lib/python3.5/site-packages/scipy/spatial/distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [27]:
len(similar_users)

200

In [47]:
##Item Rating prediction for user
###INCORRECT - need to check the formula
item_ratings = np.zeros((len(mat[0])))
for i in range(len(similar_users)):
    u = mat[similar_users[i]]
    for j in range(len(u)):
        if(math.isnan(u[j])):
            continue
        else:
            item_ratings[j] += similarity[i] * u[j]

In [49]:
##Items recommended to user based on rating prediction
c = 0
for i in range(len(item_ratings)):
    if item_ratings[i]>0:
        c = c +1
print("count is:", c)

count is: 131


In [50]:
###Item Recommendation to users based on similar_users####
ind = similar_users[1]
recommend_items_indices = []

for i in range(len(similar_users)):
    for k,v in enumerate(mat[similar_users[i]]):
        if v>0:
            recommend_items_indices.append(k)

##Removing indices which are already used by user
for k,v in enumerate(mat[320]):
    if v > 0:
        recommend_items_indices.remove(k)
            
#Recommended items is
recommended_items = []

for i in range(len(recommend_items_indices)):
    recommended_items.append(items[recommend_items_indices[i]])


In [51]:
## recommended_items - user_items
len(recommended_items)

531

In [30]:
df_bus[df_bus['business_id'] == recommended_items[0]]

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
33553,"3015 S Valley View Blvd, Ste 100",{'BusinessAcceptsCreditCards': True},MPGrHMnDFzXCr-EtNSDh2g,"[Automotive, Oil Change Stations, Smog Check S...",Las Vegas,"{'Monday': '7:30-17:00', 'Thursday': '7:30-17:...",1,36.136304,-115.191512,Bob's Autodynamics,Westside,89102,15,4.0,NV
